In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm;
import sklearn
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
train =  pd.read_csv('data/EDA/train_EDA_0910.csv')
test =  pd.read_csv('data/EDA/test_EDA_0910.csv')
feat_class = pd.read_csv('data/EDA/feat_class_0910.csv')

In [ ]:
feature_raw = feat_class['raw'].dropna().values.tolist()
feature_dis = feat_class['dis'].dropna().values.tolist()
feature_map = feat_class['map'].dropna().values.tolist()
feature_onehot = feat_class['onehot'].dropna().values.tolist()
feature_extra = feat_class['extra'].dropna().values.tolist()
feature_del = feat_class['del'].dropna().values.tolist()

# delete

In [ ]:
feature_del

In [ ]:
train = train.drop(columns = feature_del)
test = test.drop(columns = feature_del)

# Extra

In [ ]:
feature_extra

In [ ]:
train.groupby('Neighborhood').mean().sort_values(by='SalePrice')['SalePrice']

In [ ]:
map_Nei={'MeadowV' :0,
'IDOTRR'  :1,
'BrDale'  :2,
'BrkSide' :3,
'Edwards' :4,
'OldTown' :5,
'Sawyer'  :6,
'Blueste' :7,
'SWISU'   :8,
'NPkVill' :9,
'NAmes'   :10,
'Mitchel' :11,
'SawyerW' :12,
'NWAmes'  :13,
'Gilbert' :14,
'Blmngtn' :15,
'CollgCr' :16,
'Crawfor' :17,
'ClearCr' :18,
'Somerst' :19,
'Veenker' :20,
'Timber'  :21,
'StoneBr' :22,
'NridgHt' :23,
'NoRidge' :24}

In [ ]:
train['Neighborhood']=train['Neighborhood'].map(map_Nei)
test['Neighborhood']=test['Neighborhood'].map(map_Nei)

In [ ]:
train['YearOld'] = (train['YearBuilt'] + train['YearRemodAdd'] )/2
train['YearOld'] -= train['YrSold']
test['YearOld'] = (test['YearBuilt'] + test['YearRemodAdd'] )/2
test['YearOld'] -= test['YrSold']

In [ ]:
#Extra에서 따로 계산하고 지워줘야 할 컬럼 추가
train = train.drop(columns = ['YearBuilt', 'YearRemodAdd',
                    'Exterior1st', 'Exterior2nd', 'YrSold', 'GarageYrBlt'])
test = test.drop(columns = ['YearBuilt', 'YearRemodAdd',
                    'Exterior1st', 'Exterior2nd', 'YrSold', 'GarageYrBlt'])

In [ ]:
feature_dis.extend(['Neighborhood', 'MoSold'])
feature_extra=[]
feature_raw.append('YearOld')

# onehot

In [ ]:
feature_onehot

In [ ]:
from sklearn.preprocessing import LabelEncoder
LE1=LabelEncoder()
for i in feature_onehot:
    temp=LE1.fit_transform(train[i])
    train[i]=temp
    
LE2=LabelEncoder()
for i in feature_onehot:
    temp=LE2.fit_transform(test[i])
    test[i]=temp

train.column

# rename feature

In [ ]:
# 모델링 할 때 컬럼명이 숫자로 시작해서 오류가 생겼음
train.rename(columns = {'3SsnPorch' : 'SsnPorch'}, inplace = True)
test.rename(columns = {'3SsnPorch' : 'SsnPorch'}, inplace = True)

feature_raw.remove('3SsnPorch')
feature_raw.append('SsnPorch')

In [ ]:
train['SsnPorch'].value_counts()

# Scaling

In [ ]:
MMS_list=[]
MMS_list.extend(feature_raw)
MMS_list.extend(feature_dis)
MMS_list.extend(feature_map)
MMS_list.extend(feature_onehot)

In [ ]:
MMS_list

In [ ]:
from sklearn.preprocessing import MinMaxScaler
MMS1=MinMaxScaler()
train[list_MMS]=MMS1.fit_transform(train[list_MMS])

MMS2=MinMaxScaler()
test[list_MMS]=MMS2.fit_transform(test[list_MMS])

In [ ]:
train['SalePrice'] = np.log1p(train['SalePrice'])

# Save

In [ ]:
train.to_csv('data/preprocess/train_0918.csv', index = False)
test.to_csv('data/preprocess/test_0918.csv', index = False)

In [ ]:
feature_dict = {'raw':feature_raw, 'dis':feature_dis, 
               'map':feature_map, 'onehot':feature_onehot,
                'extra':feature_extra, 'del':feature_del
               }

#features = pd.DataFrame(feature_dict)
features = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in feature_dict.items() ]))
features.to_csv('data/EDA/feat_class_0918.csv', index = False)